In [ ]:
import logging
from autologging import logged, traced, TRACE
logging.basicConfig(level=logging.INFO, format="%(name)s.%(funcName)s\t%(message)s")

import tensorflow as tf
from tensorflow import keras
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()
train_images = train_images[:len(train_images)//1]
test_images = test_images[:len(test_images)//1]
logging.info(f"train_images: {train_images.shape} {train_images.dtype}")

In [ ]:
import numpy as np

@logged
def prepare_images(img_array:np.ndarray, sz=64):
    from skimage.transform import resize
    img_array = (img_array / 255.0).astype(np.float32)
    prepare_images._log.info(f"img_array: {img_array.shape} {img_array.dtype}")    
    img_array = [resize(img_array[n], (sz,sz)) for n in range(img_array.shape[0])]
    img_array = np.reshape(img_array, (len(img_array),sz,sz,1))
    prepare_images._log.info(f"img_array: {img_array.shape} {img_array.dtype}")
    return img_array

train_images = prepare_images(train_images)
test_images = prepare_images(test_images)

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from custom_layers import GaborPowerMap2D, LogSumExpPooling2D

def create_encoder(sz=64, latent_dim=8, locally_connected_channels=2, act_func='softplus',
        gabor_powermaps=True, logsumexp_pool=True):
    """ creates the encoder side of the autoencoder, for the parameters sz and latent_dim
    # Static parameters
        sz (int): sz x sz input
        latent_dim (int): gaussian dimensions
        locally_connected_channels = 2
    # Arguments
        <none>
    # Returns
        retina: the input layer
        encoder: the encoder model
        shape: shape of last input layer
        [z_mean, z_log_var, z]: tensors for latent space
    """

    return Sequential([
        Input(shape=(sz,sz,1), name='retina_{}'.format(sz)),

        ##### V1 layers

        GaborPowerMap2D(16, (5,5), name='v1_powmap') 
        if gabor_powermaps 
        else Conv2D(16, (5,5), name='v1_conv2d', activation=act_func, padding='same'),

        LogSumExpPooling2D(name='v1_pool') 
        if logsumexp_pool 
        else MaxPooling2D((2,2), name='v1_maxpool', padding='same'),

        SpatialDropout2D(0.1, name='v1_dropout'),

        ##### V2 layers

        GaborPowerMap2D(16, (3,3), name='v2_powmap') 
        if gabor_powermaps 
        else Conv2D(16, (3,3), name='v2_conv2d', activation=act_func, padding='same'),

        LogSumExpPooling2D(name='v2_pool') 
        if logsumexp_pool 
        else MaxPooling2D((2,2), name='v2_maxpool', padding='same'),

        ##### V4 layers

        GaborPowerMap2D(32, (3,3), name='v4_powmap') 
        if gabor_powermaps 
        else Conv2D(32, (3,3), name='v4_conv2d', activation=act_func, padding='same'),

        LogSumExpPooling2D(name='v4_pool') 
        if logsumexp_pool 
        else MaxPooling2D((2,2), name='v4_maxpool', padding='same'),

        ##### IT Layers

        Conv2D(32, (3,3), name='pit_conv2d', activation=act_func, padding='same'),
        Conv2D(64, (3,3), name='cit_conv2d', activation=act_func, padding='same'),
        LocallyConnected2D(locally_connected_channels, (3,3), name='ait_local',        
            activation=act_func),
        ActivityRegularization(l1=0.0e-4, l2=0.0e-4, name='ait_regular'),

        ##### Pulvinar

        # generate latent vector Q(z|X)
        Flatten(name='pulvinar_flatten'),
        Dense(latent_dim, name='pulvinar_dense', activation=act_func),
        Dense(latent_dim+latent_dim, name='z_mean_log_var'),
    ], name='v1_to_pulvinar_encoder')

encoder = create_encoder()
encoder.summary()

retina = encoder.input
print(retina)

dense_shape = encoder.get_layer('ait_local').output_shape
print(dense_shape)

output = encoder.get_layer('z_mean_log_var')
print(output)

In [ ]:
def create_decoder(dense_shape, latent_dim=8, locally_connected_channels=2,
        act_func='softplus'):
    """ creates the decoder side of the autoencoder, given the input shape
    # Static parameters
        sz (int): sz x sz input
        latent_dim (int): gaussian dimensions
        locally_connected_channels = 2
    # Arguments
        shape: first input layer shape
    # Returns
        decoder: the decoder model
    """

    return Sequential([
        Input(shape=(latent_dim,), name='z_sampling'),
        Dense(dense_shape[1] * dense_shape[2] * dense_shape[3], 
            name='pulvinar_dense_back', activation=act_func),
        Reshape((dense_shape[1], dense_shape[2], dense_shape[3]),
            name='pulvinar_antiflatten'),

        ##### IT retro Layers

        ZeroPadding2D(padding=(1,1), name='ait_padding_back'),
        LocallyConnected2D(locally_connected_channels, (3,3), name='ait_local_back',
            activation=act_func),

        ZeroPadding2D(padding=(1,1), name='cit_padding_back'),
        Conv2DTranspose(64, (3,3), name='cit_conv2d_trans', 
                activation=act_func, padding='same'),
        Conv2DTranspose(32, (3,3), name='pit_conv2d_trans', 
                activation=act_func, padding='same'),

        #### V4 retro layers

        Conv2DTranspose(32, (3,3), name='v4_conv2d_trans', 
                activation=act_func, padding='same'),
        UpSampling2D((2,2), name='v4_upsample_back'),

        #### V2 retro layers

        Conv2DTranspose(16, (3,3), name='v2_conv2d_trans', 
                activation=act_func, padding='same'),
        UpSampling2D((2,2), name='v2_upsample_back'),

        #### V1 retro layers

        Conv2D(1, (5,5), name='v1_conv2d_5x5_back', 
                            # activation='sigmoid', no sigmoid == return logits
                            padding='same'),
        UpSampling2D((2,2), name='v1_upsample_back'),
    ], name='pulvinar_to_v1_decoder')

decoder = create_decoder(dense_shape, latent_dim=8)
decoder.summary()

In [ ]:
# @traced
def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(
        -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
        axis=raxis)

# @traced
def reparameterize(mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

# @traced
def compute_loss(encoder, decoder, x):
    mean, logvar = tf.split(encoder(x), num_or_size_splits=2, axis=1)
    z = reparameterize(mean, logvar)
    x_logit = decoder(z)
    # tf.print(f"x_logit, x.shape = {x_logit}, {x.shape}")
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
    # tf.print(f"cross_ent.shape = {cross_ent.shape}")
    logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
    logpz = log_normal_pdf(z, 0., 0.)
    logqz_x = log_normal_pdf(z, mean, logvar)
    return -tf.reduce_mean(logpx_z + logpz - logqz_x)

@tf.function
def train_step(encoder, decoder, x, optimizer):
    """Executes one training step and returns the loss.
    This function computes the loss and gradients, and uses the latter to
    update the model's parameters.
    """
    all_trainable_variables = encoder.trainable_variables + decoder.trainable_variables
    with tf.GradientTape() as tape:
        loss = compute_loss(encoder, decoder, x)
        # tf.print(f"loss value = {loss}")
    gradients = tape.gradient(loss, all_trainable_variables)
    optimizer.apply_gradients(zip(gradients, all_trainable_variables))

from typing import Callable

def forall_batch(input_data:np.ndarray, batch_size:int, 
            train_func:Callable[[np.ndarray],None],
            tb_callback=None):
    """Batches and trains using a given function
    """
    start_time = time()
    step = 0
    while (step+1) * batch_size < input_data.shape[0]:
        input_batch = input_data[step*batch_size:(step+1)*batch_size,...]
        if not (step % 10000): 
            logging.info(f"{step}: {input_batch.shape}")
        if tb_callback: 
            tb_callback.on_train_batch_begin(step, logs=logs)

        train_func(input_batch)

        if tb_callback: 
            tb_callback.on_train_batch_end(step, logs=logs)
        step += 1
    end_time = time()
    return end_time - start_time



In [ ]:
from time import time
from datetime import datetime
from pathlib import Path
log_dir = Path('.') / 'logs' / 'fit' / datetime.now().strftime("%Y%m%d-%H%M%S")
print(log_dir)

tb_callback = None # tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
if tb_callback:
    tb_callback.set_model(encoder)
    logs = { 'loss':None, 'mean_absolute_error':None, 'output':None }
    tb_callback.on_train_begin()

batch_size = 16
optimizer = tf.keras.optimizers.Adam(1e-4)

for epoch in range(1, 121):
    if tb_callback: 
        tb_callback.on_epoch_begin(epoch)

    # perform training
    time_elapsed = forall_batch(train_images, 16, 
                        lambda batch: train_step(encoder, decoder, batch, optimizer))

    # calculate test results
    loss = tf.keras.metrics.Mean()
    forall_batch(test_images, 16, 
        lambda batch: loss(compute_loss(encoder, decoder, input_test_x)))
    elbo = -loss.result()
    logging.info(f"Epoch: {epoch}, test ELBO: {elbo}, time elapsed: {end_time - start_time}")

    if tb_callback:
         tb_callback.on_epoch_end(epoch)

if tb_callback: 
    tb_callback.on_train_end()

In [ ]:
import matplotlib.pyplot as plt
test_latent = encoder(test_images[0:10,...])

def sigmoid(x):
    return np.exp(-np.logaddexp(0, -x))

re_test_images = decoder(test_latent[...,0:8])
re_test_images = sigmoid(re_test_images)

quantiles = [0.05, 0.95]

fig, ax = plt.subplots(2,10,figsize=(15,3))
for n in range(10):
    q_test = np.quantile(test_images[n],quantiles)
    ax[0][n].imshow(test_images[n], cmap='gray', vmin=q_test[0], vmax=q_test[1])

    reshape_test_image = np.reshape(re_test_images[n], (64,64))
    q_re = np.quantile(reshape_test_image,quantiles)
    ax[1][n].imshow(reshape_test_image, cmap='gray', vmin=q_re[0], vmax=q_re[1])